In [1]:
import os
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.llms.lmstudio import LMStudio
import json
import dotenv
import phoenix as px
from llama_index.core import (
    Settings,
    set_global_handler,
)
import nest_asyncio
from phoenix.evals import (
    HallucinationEvaluator,
    OpenAIModel,
    QAEvaluator,
    RelevanceEvaluator,
    run_evals,
)
from phoenix.session.evaluation import (
    get_qa_with_reference,
    get_retrieved_documents,
)
from phoenix.trace import DocumentEvaluations, SpanEvaluations
from tqdm import tqdm
from llama_index.llms.bedrock import Bedrock
from llama_index.llms.bedrock_converse import BedrockConverse
from llama_index.embeddings.bedrock import BedrockEmbedding

from llama_index.readers.file import PyMuPDFReader, PDFReader
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
import unicodedata
import boto3
import json
from datetime import datetime
import re
from pydrive2.fs import GDriveFileSystem
from llama_index.readers.file import DocxReader


dotenv.load_dotenv()
nest_asyncio.apply()

In [2]:
# pip install llama-index-llms-bedrock
# !pip install llama-index-llms-bedrock-converse

In [3]:
# session = px.launch_app()
# set_global_handler("arize_phoenix")

In [4]:
# Create a Bedrock Runtime client in the AWS Region of your choice.
client = boto3.client(
    "bedrock-runtime",
    region_name=os.environ.get("AWS_REGION"),
    aws_access_key_id=os.environ.get("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key=os.environ.get("AWS_SECRET_ACCESS_KEY")
)

In [5]:
# pip install llama-index-embeddings-bedrock

In [6]:
llm = BedrockConverse(
    model="amazon.nova-micro-v1:0",
    client=client,
    region_name="us-east-1",
)
embedder = BedrockEmbedding(
    model_name="amazon.titan-embed-text-v2:0",
    client=client,
    region_name="us-east-1",
)

Settings.llm = llm
Settings.embed_model = embedder

In [7]:
llm.metadata.is_function_calling_model = True

In [8]:
llm.metadata

LLMMetadata(context_window=128000, num_output=512, is_chat_model=True, is_function_calling_model=True, model_name='amazon.nova-micro-v1:0', system_role=<MessageRole.SYSTEM: 'system'>)

In [9]:
llm.complete("hello")

CompletionResponse(text="Hello! How can I assist you today? Whether you have a question, need information, or just want to chat, I'm here to help.", additional_kwargs={'tool_calls': [], 'tool_call_id': [], 'status': []}, raw={'ResponseMetadata': {'RequestId': 'a48578b9-12e1-4076-bf6c-6c4185df8e72', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Thu, 27 Feb 2025 09:06:13 GMT', 'content-type': 'application/json', 'content-length': '301', 'connection': 'keep-alive', 'x-amzn-requestid': 'a48578b9-12e1-4076-bf6c-6c4185df8e72'}, 'RetryAttempts': 0}, 'output': {'message': {'role': 'assistant', 'content': [{'text': "Hello! How can I assist you today? Whether you have a question, need information, or just want to chat, I'm here to help."}]}}, 'stopReason': 'end_turn', 'usage': {'inputTokens': 1, 'outputTokens': 31, 'totalTokens': 32}, 'metrics': {'latencyMs': 305}}, logprobs=None, delta=None)

In [10]:
embedder.get_text_embedding("hello")

[-0.05515049770474434,
 0.04516850784420967,
 0.02720092423260212,
 0.006394712720066309,
 0.04142526164650917,
 -0.0359351672232151,
 0.009233341552317142,
 0.0314432717859745,
 0.017094159498810768,
 0.010793027468025684,
 -0.009794828481972218,
 0.03718291595578194,
 0.007611267734318972,
 -0.05415229871869087,
 -0.011104964651167393,
 -0.03543606773018837,
 -0.014972985722124577,
 -0.035685617476701736,
 0.04766400530934334,
 0.002916612895205617,
 -0.03393876925110817,
 0.016220735386013985,
 0.03393876925110817,
 -0.020587855949997902,
 -0.027700023725628853,
 -0.02607795037329197,
 -0.035186517983675,
 -0.03543606773018837,
 0.00948289129883051,
 -0.05714689567685127,
 0.04741445556282997,
 -0.0070497808046638966,
 -0.008235142566263676,
 -0.01397478673607111,
 0.0160959605127573,
 0.03793156519532204,
 0.019963981583714485,
 -0.02345767803490162,
 -0.012352713383734226,
 -0.01647028513252735,
 0.05440184846520424,
 -0.028199123218655586,
 -0.0061139692552387714,
 0.021086955443

In [9]:
documents = SimpleDirectoryReader(input_files=["data/Phone_and_Internet_Allowance_Policy_abrd.pdf"]).load_data()

In [10]:
index = VectorStoreIndex.from_documents(documents, embed_model=embedder)
query_engine = index.as_query_engine(llm=llm, similarity_top_k=3)

In [11]:
response = query_engine.query("What does the term 'Social Media' refer to in the document?")
print(response)

In a professional setting, social media typically refers to online platforms where individuals or organizations share information, ideas, and experiences with a wide audience. These platforms often enable users to create and manage their own content, engage with others through comments and messages, and build relationships with people they may not know personally.


In [12]:
for source in response.source_nodes:
    print(source, end="\n")

Node ID: baa65c3d-7b9d-4e5b-bfb9-af0276bd2901
Text: EMPLOYEE  COMMUNICATIONS  POLICY    Document  owner:
Reference:    Author:   Communications  and  Human  Resources   Date:
14/08/2024   Approved  By:    Head  Human  Resources    Approval
Date:   14/01/2025   Classification:   Internal   Version:  1.0
Effective  Date:  14/01/2025    Record  of  Change   Name  Designation
Reason  for  Ch...
Score:  0.655

Node ID: 32e97649-cfac-427e-b904-a5b5e242638c
Text: 1.2.5  Members  of  employees  are  to  always  treat  others
with  respect  on  social  media.  If  someone  questions  or
criticizes   the   work   we   do   at   Helium   Health,   responses
should   be   thoughtful   and   kind.   If   the   criticism   is
especially   aggressive   or   inconsiderate,   it   is   recommended
that   ...
Score:  0.646

Node ID: ef8b46a3-3c17-4cde-953d-4a28ca43cbcc
Text:
Score:  0.631



In [13]:
response = query_engine.query("List some social media guidelines for usage?")
print(response)

for source in response.source_nodes:
    print(source, end="\n")

Here are some social media guidelines for usage:

1. Be ambassadors of the company by being respectful and professional online.
2. Employees are responsible for the content they put out or curate on their personal social media accounts, even if they are not affiliated with the company.
3. Avoid posting or sharing inflammatory, violent, or spreading misinformation.
4. Treat others with respect on social media, especially when responding to criticism of the company's work.
5. Refrain from responding to aggressive or inconsiderate comments.
6. Be mindful that personal social media accounts are public spaces and actions taken on them may have repercussions at work.
Node ID: 32e97649-cfac-427e-b904-a5b5e242638c
Text: 1.2.5  Members  of  employees  are  to  always  treat  others
with  respect  on  social  media.  If  someone  questions  or
criticizes   the   work   we   do   at   Helium   Health,   responses
should   be   thoughtful   and   kind.   If   the   criticism   is
especially   aggr

In [14]:
response = query_engine.query("What are professional websites?")
print(response)

for source in response.source_nodes:
    print(source, end="\n")

Websites that provide legitimate online services or resources for job or professional growth.
Node ID: 32e97649-cfac-427e-b904-a5b5e242638c
Text: 1.2.5  Members  of  employees  are  to  always  treat  others
with  respect  on  social  media.  If  someone  questions  or
criticizes   the   work   we   do   at   Helium   Health,   responses
should   be   thoughtful   and   kind.   If   the   criticism   is
especially   aggressive   or   inconsiderate,   it   is   recommended
that   ...
Score:  0.532

Node ID: bb36f5b9-4f31-4214-9ea2-9bee37aa12fe
Text: 4.2.8.  Helium  Health  understands  the  benefits  of  a
diverse  workforce  and  expects  that  every  employee  should
foster   an   understanding   of   others’   differences   in   order
to   create   an   environment   where   those   differences
contribute   to   a   better   organization.    5.  COMPANY  EMAIL
USE  5.1.  Definiti...
Score:  0.517

Node ID: d00f1d5b-d6ac-4428-8f96-2a01dddf33fc
Text: 5.5.  Email  Etiquette   5.5.1.  Al

## PDF Readers

In [15]:
# pip install PyMuPDF

In [16]:
documents[2]

Document(id_='14c9dc1d-296e-47e4-9b06-e108ed79ffde', embedding=None, metadata={'page_label': '3', 'file_name': 'Employee Communication Policy v1.pdf', 'file_path': 'data/Employee Communication Policy v1.pdf', 'file_type': 'application/pdf', 'file_size': 229997, 'creation_date': '2025-01-22', 'last_modified_date': '2025-01-21'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text=" \nThis  Employee  Communication  Policy  (“ Policy ”)  contains  the  following  sections:   1.  SOCIAL  MEDIA  GUIDELINES   2.  PROFESSIONAL  WEBSITES  3.  THOUGHT  LEADERSHIP  PRACTICES  4.  INTERNAL  COMMUNICATION  PLATFORMS   5.  COMPANY  EMAIL  USE   1.  SOCIAL  

In [70]:
# PyMuPDF Reader example
parser = PyMuPDFReader()
file_extractor = {".pdf": parser}
documents = SimpleDirectoryReader(
    input_files=["./data/transport allowance.pdf"], file_extractor=file_extractor, encoding="utf8"
).load_data()

# PDF Reader with `SimpleDirectoryReader`
# parser = PDFReader()
# file_extractor = {".pdf": parser}
# documents = SimpleDirectoryReader(
#     "./data", file_extractor=file_extractor
# ).load_data()

In [75]:
documents[0].text

'One Global Medical Technology Ltd.\nRC: 1264820\nMEMO\nTO: Chief Operating Officer (COO), Finance Team, HR Team\nFROM: Head Human Resources\nDATE: 24th August 2023\nSUBJECT: Lagos Team Transport Allowance (Update on Travel Policy for Lagos Team Members)\nThis memo is being developed in response to the recent hike in transportation and petrol costs in Nigeria\nand is to apply to Helium Team Members who operate hybrid workdays of a minimum of 2 days in the\noffice. As at the time of this memo, this will apply to Lagos Team members only who are not receiving any\nkind of transport allowance. This memo is effective on the date stated, transport costs to the office within\nLagos will be subsidised by the Company at a monthly flat fee rate.\nPolicy Statement.\n●\nHelium Health will pay a Transport Allowance as a subsidised allowance for Lagos based employees\nonly, who are not receiving any kind of transport allowance.\n●\nAn allowance for transportation for employees and for personal use h

In [76]:
normalized = unicodedata.normalize("NFKC", documents[0].text).strip().replace('\n', '').strip()
normalized

'One Global Medical Technology Ltd.RC: 1264820MEMOTO: Chief Operating Officer (COO), Finance Team, HR TeamFROM: Head Human ResourcesDATE: 24th August 2023SUBJECT: Lagos Team Transport Allowance (Update on Travel Policy for Lagos Team Members)This memo is being developed in response to the recent hike in transportation and petrol costs in Nigeriaand is to apply to Helium Team Members who operate hybrid workdays of a minimum of 2 days in theoffice. As at the time of this memo, this will apply to Lagos Team members only who are not receiving anykind of transport allowance. This memo is effective on the date stated, transport costs to the office withinLagos will be subsidised by the Company at a monthly flat fee rate.Policy Statement.●Helium Health will pay a Transport Allowance as a subsidised allowance for Lagos based employeesonly, who are not receiving any kind of transport allowance.●An allowance for transportation for employees and for personal use has been provided under the“Transpo

In [46]:
re.split(r'[ ]{3,}', normalized)

['3.6. An employee whose device either becomes fully or partially inoperative or is otherwise unusable shall attempt to',
 'repair or replace their device or notify the HR Team within 5 days through any appropriate and available mode',
 'of contact.  4.PROCESSING 4.1. The department head will determine those employees in their department for whom the Phone and / or Internet',
 'Allowance is justified and the level of need for that employee. To request the allowance the department head',
 'will send the list of eligible employees to HR indicating the usage level.',
 '4.2. The employee will receive a Phone Allowance in the form of credit purchase to be paid monthly by the Finance',
 'department directly to the appropriate network providers; and Internet Allowance to be paid to the employee. 4.3. This allowance would come in before the end of the first week of every new month. 4.4. This allowance does not constitute an increase to base pay and will not be included in the calculation of',


In [97]:
# replace `root` with ID of a drive or directory and give service account access to it
fs = GDriveFileSystem("1a59en4nkmKnfI7vxO7_CyMEBSmEAeNFM", client_id="",
        client_secret="")

for fnames in fs.walk("1a59en4nkmKnfI7vxO7_CyMEBSmEAeNFM/"):
    print(fnames, end="w")

('1a59en4nkmKnfI7vxO7_CyMEBSmEAeNFM', [], ['Memo_ Lagos Team Transport Allowance abridged.pdf', 'Phone_and_Internet_Allowance_Policy_abrd.pdf', 'Full_Policy_Jan_2021.pdf', 'Contract_Signatory_Authority_Policy.pdf', 'Leave_Handover_Form_shrd.docx', 'Intellectual_Property_and_Copyright_Compliance_Policy_17062022.pdf', 'Guideline_document_on_the_use_of_Generative_AI_clean_28072023.pdf', 'Helium_Health_Employee_Handbook_V1.0_20210825 (2).pdf', 'Full_Policy_updated_Aug_2023 (2).pdf', 'Memo__Meeting_and_Email_Guidelines_2023.pdf', 'HH_ISMS0502_POL_Information_Security_Management_System_Policy_V3.0_20240710_FIN (1).docx', 'HH_ISMSA.5.1_POL_Policies_for_Information_Security_V3.0_20240820_FIN (1).docx', 'Org Structure shared Nov 2024.pdf', 'Employee Communication Policy v1.pdf'])w

In [117]:
reader = SimpleDirectoryReader(
    input_files=['1a59en4nkmKnfI7vxO7_CyMEBSmEAeNFM/Leave_Handover_Form_shrd.docx'],
    fs=fs,
    file_extractor={".pdf": DocxReader()}
    # recursive=True,  # recursively searches all subdirectories
)

In [118]:
docs = reader.load_data(fs=fs)

Failed to load file 1a59en4nkmKnfI7vxO7_CyMEBSmEAeNFM/Leave_Handover_Form_shrd.docx with error: File is not a zip file. Skipping...


In [111]:
docs[0]

Document(id_='ada6e74d-2c4a-4e20-889b-4453198de206', embedding=None, metadata={'page_label': '1', 'file_name': 'Memo_ Lagos Team Transport Allowance abridged.pdf', 'file_path': '1a59en4nkmKnfI7vxO7_CyMEBSmEAeNFM/Memo_ Lagos Team Transport Allowance abridged.pdf', 'file_type': 'application/pdf', 'file_size': 109667}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='OneGlobal Medical TechnologyLtd.RC:1264820\nMEMO\nTO: Chief OperatingOfficer(COO), FinanceTeam, HRTeamFROM: HeadHumanResourcesDATE: 24thAugust2023SUBJECT: LagosTeamTransportAllowance(UpdateonTravel PolicyforLagosTeamMembers)\nThis memois being developedinresponse tothe recent hike

## Clean Text 

In [9]:
def clean_text(text):
    # normalize text for unicode
    clean_text = unicodedata.normalize("NFKD", text).strip()
    clean_text = clean_text.replace('\n', ' ')
    clean_text = " ".join(re.split(r'[ ]{3,}', clean_text))
    return clean_text


In [25]:
clean_text(documents[0].text)

'RECRUITMENT AND SELECTION HR POLICY This Policy includes:  1. Introduction 2. Workforce Planning  3. Job Descriptions 4. Recruitment Strategy 5. Recruitment Process  6. Recruitment Closing  7. Employment of Close Relatives  8. Employment of Returnees / Former Employees 1. Introduction 1.1. Policy Objective  1.1.1. To facilitate Helium Health’s commitment to hiring highly competent and talented resources who are the best fit for available positions; to equal employment opportunities; to supporting all new hires to make the best use of their abilities for the good of the company. 1.2. Policy Statements  1.2.1. Recruitment and Selection Policies will support the strategic priorities of all Helium Health teams. All recruitment activities must be duly approved and in line with the workforce plan and organisational requirements.  1.2.2. Helium Health is committed to strengthening its capability, performance as well as corporate image by attracting and recruiting qualified and skilled employ

## Document Splitters

In [20]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(
    chunk_size=256,
    chunk_overlap=10,
)

print(f"Len documents: ", len(documents))
nodes = splitter.get_nodes_from_documents(documents)
print(f"Len nodes: ", len(nodes))

Len documents:  7
Len nodes:  18


In [21]:
print(nodes[4])

Node ID: 858eb49e-695b-42d6-b42b-8669725369d8
Text: 1.2.4​ Employees' personal social media accounts are online safe
zones and are not policed by Helium Health. It is  however the duty of
the employee to remember that these are very public spaces and actions
taken on these  spaces may have repercussions at Helium Health if they
are considered to be egregious.  1.2.5​ Members of employees are to
a...


In [22]:
index = VectorStoreIndex(nodes=nodes, embed_model=embedder)
query_engine = index.as_query_engine(llm=llm, similarity_top_k=5)

In [23]:
response = query_engine.query("What does the term 'Social Media' refer to in the document?")
print(response, end="\n")

for source in response.source_nodes:
    print(source, end="\n")

The term "Social Media" refers to online platforms which aid interactions among people, including but not limited to X (formerly Twitter), Facebook, Tiktok, Snapchat, or any other platform by which individuals engage for the purpose(s) of creating or curating content.
Node ID: 9193f6d4-133b-431f-afdb-c513aefb9ffa
Text: This Employee Communication Policy (“Policy”) contains the
following sections:    1.​ SOCIAL MEDIA GUIDELINES   2.​ PROFESSIONAL
WEBSITES  3.​ THOUGHT LEADERSHIP PRACTICES  4.​ INTERNAL COMMUNICATION
PLATFORMS   5.​ COMPANY EMAIL USE    1.​ SOCIAL MEDIA GUIDELINES
1.1.​ Definitions  1.1.1.​ The usage of the term “Social Media” now
and hitherto...
Score:  0.748

Node ID: b42b40ac-b7a7-41d5-b38b-5c3facd82e14
Text: If  you provide us with any contact information for customer
service purposes, it will only be used to  manage your enquiry and
will not be used for any other reason.”  OR  “Any personal data shared
in public by the user on social platforms is shared at their own

In [24]:
response = query_engine.query("List some social media guidelines for usage?")
print(response, end="\n\n")

for source in response.source_nodes:
    print(source, end="\n")

The company encourages members of the team to be ambassadors of the company on social media platforms. When engaging online, employees are viewed by customers, partners, and other external parties, placing an even greater responsibility on them.

Employees are responsible for the content they put out or curate on these platforms, and are prohibited from posting or sharing anything that may be considered inflammatory, inciting of violence (in any form) or spreading of misinformation.

Node ID: 9193f6d4-133b-431f-afdb-c513aefb9ffa
Text: This Employee Communication Policy (“Policy”) contains the
following sections:    1.​ SOCIAL MEDIA GUIDELINES   2.​ PROFESSIONAL
WEBSITES  3.​ THOUGHT LEADERSHIP PRACTICES  4.​ INTERNAL COMMUNICATION
PLATFORMS   5.​ COMPANY EMAIL USE    1.​ SOCIAL MEDIA GUIDELINES
1.1.​ Definitions  1.1.1.​ The usage of the term “Social Media” now
and hitherto...
Score:  0.768

Node ID: b42b40ac-b7a7-41d5-b38b-5c3facd82e14
Text: If  you provide us with any contact informa

In [25]:
response = query_engine.query("What are professional websites?")
print(response, end="\n\n")

for source in response.source_nodes:
    print(source, end="\n")

Professional networking platforms such as LinkedIn, Angel List, Crunchbase, and others that allow individuals to share or curate content related to their work or professional progress.

Node ID: b42b40ac-b7a7-41d5-b38b-5c3facd82e14
Text: If  you provide us with any contact information for customer
service purposes, it will only be used to  manage your enquiry and
will not be used for any other reason.”  OR  “Any personal data shared
in public by the user on social platforms is shared at their own
risk.”    1.3.2​ The Admin(s) should delete (where possible) any
personal data that...
Score:  0.662

Node ID: 9193f6d4-133b-431f-afdb-c513aefb9ffa
Text: This Employee Communication Policy (“Policy”) contains the
following sections:    1.​ SOCIAL MEDIA GUIDELINES   2.​ PROFESSIONAL
WEBSITES  3.​ THOUGHT LEADERSHIP PRACTICES  4.​ INTERNAL COMMUNICATION
PLATFORMS   5.​ COMPANY EMAIL USE    1.​ SOCIAL MEDIA GUIDELINES
1.1.​ Definitions  1.1.1.​ The usage of the term “Social Media” now
and hithert

In [26]:
from llama_index.core.node_parser import SemanticSplitterNodeParser

splitter = SemanticSplitterNodeParser(
    buffer_size=1, breakpoint_percentile_threshold=95, embed_model=embedder
)

print(f"Len documents: ", len(documents))
nodes = splitter.get_nodes_from_documents(documents)
print(f"Len nodes: ", len(nodes))

Len documents:  7
Len nodes:  12


In [27]:
documents[2]

Document(id_='3f7f2aea-22c3-425f-a9fb-7cf68d4af628', embedding=None, metadata={'file_path': '/Users/sasuusen/Projects/Helium/lola/notebooks/data/Employee Communication Policy v1.pdf', 'file_name': 'Employee Communication Policy v1.pdf', 'file_type': 'application/pdf', 'file_size': 229997, 'creation_date': '2025-01-22', 'last_modified_date': '2025-01-21', 'total_pages': 7, 'source': '3'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text=" \nThis Employee Communication Policy (“Policy”) contains the following sections: \n \n1.\u200b\nSOCIAL MEDIA GUIDELINES  \n2.\u200b\nPROFESSIONAL WEBSITES \n3.\u200b\nTHOUGHT LEADERSHIP PRACTICES \n4.\u200b\

In [28]:
nodes[3]

TextNode(id_='b7430188-5e5e-4de6-85bc-2b68c4aece98', embedding=None, metadata={'file_path': '/Users/sasuusen/Projects/Helium/lola/notebooks/data/Employee Communication Policy v1.pdf', 'file_name': 'Employee Communication Policy v1.pdf', 'file_type': 'application/pdf', 'file_size': 229997, 'creation_date': '2025-01-22', 'last_modified_date': '2025-01-21', 'total_pages': 7, 'source': '3'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='3f7f2aea-22c3-425f-a9fb-7cf68d4af628', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': '/Users/sasuusen/Projects/Helium/lola/notebooks/data/Employee Communication Policy v1.pdf', 'file_name': 'Employee Communication Policy v1.pdf', 'file_type': 'application/pdf', 

In [29]:
index = VectorStoreIndex(nodes=nodes, embed_model=embedder)
query_engine = index.as_query_engine(llm=llm, similarity_top_k=5)

In [30]:
response = query_engine.query("What can i post on my personal twitter?")
print(response, end="\n")

for source in response.source_nodes:
    print(source, end="\n")

You should refrain from posting anything that may be considered inflammatory, inciting of violence, spreading of misinformation, or lacking in basic human decency.
Node ID: 13c4b871-6e93-4c9c-a478-5fab39d12db4
Text: 1.2.8​ Employees are not to share, reshare or endorse any news
about Helium Health on their social media without  confirming its
authenticity from line managers or the members of the Communications
team as they may be  false or unconfirmed.    1.3​Data Protection
1.3.1​ Where a social media user expresses their intention to share
their personal...
Score:  0.611

Node ID: ba19a985-ce17-48a0-a19a-57c882b5c000
Text: This Employee Communication Policy (“Policy”) contains the
following sections:    1.​ SOCIAL MEDIA GUIDELINES   2.​ PROFESSIONAL
WEBSITES  3.​ THOUGHT LEADERSHIP PRACTICES  4.​ INTERNAL COMMUNICATION
PLATFORMS   5.​ COMPANY EMAIL USE    1.​ SOCIAL MEDIA GUIDELINES
1.1.​ Definitions  1.1.1.​ The usage of the term “Social Media” now
and hitherto...
Score:  0.608

Nod

In [33]:
response.response

'You should refrain from posting anything that may be considered inflammatory, inciting of violence, spreading of misinformation, or lacking in basic human decency.'

## Redis VS

In [119]:
from llama_index.storage.docstore.redis import RedisDocumentStore
from llama_index.vector_stores.redis import RedisVectorStore
from llama_index.storage.index_store.redis import RedisIndexStore
from redisvl.schema import IndexSchema
from llama_index.core.storage import StorageContext

In [120]:
REDIS_HOST = "localhost"
REDIS_PORT = 6379
LLM = Ollama(model="llama3-groq-tool-use", request_timeout=60.0)
EMBED_MODEL = OllamaEmbedding(
    model_name="nomic-embed-text",
)
DOC_STORE = RedisDocumentStore.from_host_and_port(
    REDIS_HOST, REDIS_PORT, namespace="lola_document_store"
)
VECTOR_STORE = RedisVectorStore(
    schema=IndexSchema.from_dict(dict(json.load(open("../custom_redis_vector_schema.json", "r")))),
    redis_url=f"redis://{REDIS_HOST}:{REDIS_PORT}",
)
INDEX_STORE = RedisIndexStore.from_host_and_port(
    host=REDIS_HOST, port=REDIS_PORT, namespace="lola_index"
)
STORAGE_CONTEXT = StorageContext.from_defaults(
    index_store=INDEX_STORE,
    docstore=DOC_STORE,
)

14:46:08 redisvl.index.index INFO   Index already exists, not overwriting.


In [150]:
from llama_index.core import load_index_from_storage, load_indices_from_storage

In [151]:
vector_index = load_indices_from_storage(
    STORAGE_CONTEXT
)

In [160]:
vector_index[2].summary

'None'

In [161]:
vector_engine = vector_index[1].as_retriever(similarity_top_k=3)

In [162]:
await vector_engine.aretrieve("phone and internet policy?")

[]

## Evaluation

In [18]:
from llama_index.llms.lmstudio import LMStudio

In [19]:
from phoenix.evals import LiteLLMModel
from litellm import completion

/Users/sasuusen/miniconda3/envs/Helium/lib/python3.12/site-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


In [20]:
import os

os.environ['LM_STUDIO_API_BASE'] = "http://127.0.0.1:1234/v1"
os.environ['LM_STUDIO_API_KEY'] = "sk-123"

In [21]:
eval_model = LiteLLMModel(model="lm_studio/phi4-latest", _verbose=True)

In [22]:
response = eval_model("Hey there, what is 2+2?")
print(str(response))

2 + 2 equals 4. If you have any more questions or need further assistance, feel free to ask!


In [23]:
queries_df = get_qa_with_reference(px.Client())
retrieved_documents_df = get_retrieved_documents(px.Client())

In [24]:
queries_df

,input,output,reference
context.span_id,,,
78c28db07ff9886c,What does the term 'Social Media' refer to in ...,"In the document, ""social media"" refers to onli...",EMPLOYEE COMMUNICATIONS POLICY Document ...
2afccf2582b4cae4,List some social media guidelines for usage?,The organization encourages team members to ac...,1.2.5 Members of employees are to always...
b886d4efd22d34fb,What are professional websites?,Professional websites refer to online platform...,1.2.5 Members of employees are to always...
35fab9bee5eb6300,What does the term 'Social Media' refer to in ...,"The term ""Social Media"" in the document refers...",Definitions 1.1.1. The usage of the term...
3354bc4371077c9f,List some social media guidelines for usage?,Employees are encouraged to act as ambassadors...,Definitions 1.1.1. The usage of the term...
1e5c387c5d636d65,What are professional websites?,Professional websites refer to platforms desig...,Definitions 2.1.1. The usage of the term...


In [25]:
retrieved_documents_df

context.trace_id  \
context.span_id  document_position                                     
530af4522d6d8273 0                  d51ccb08c0fedf13cb7faadc0ef8709b   
                 1                  d51ccb08c0fedf13cb7faadc0ef8709b   
                 2                  d51ccb08c0fedf13cb7faadc0ef8709b   
fbefcb4215e8cdc8 0                  66a9aafd70cbf0ad6fbedfc2ac36a95b   
                 1                  66a9aafd70cbf0ad6fbedfc2ac36a95b   
                 2                  66a9aafd70cbf0ad6fbedfc2ac36a95b   
ab83ed62670696ff 0                  9d6a962b94adafa8b989b7b5939b443c   
                 1                  9d6a962b94adafa8b989b7b5939b443c   
                 2                  9d6a962b94adafa8b989b7b5939b443c   
fed4ecdba15db354 0                  bf17d1e72b5ff31b1e0d6cf680e10b0f   
                 1                  bf17d1e72b5ff31b1e0d6cf680e10b0f   
                 2                  bf17d1e72b5ff31b1e0d6cf680e10b0f   
                 3                  bf17d1e72b5ff31b1e0d6cf680e10b0f   
                 4                  bf17d1e72b5ff31b1e0d6cf680e10b0f   
aa15c5e3b65359af 0                  de3849188cb84f02bf780b556c9b02eb   
                 1                  de3849188cb84f02bf780b556c9b02eb   
                 2                  de3849188cb84f02bf780b556c9b02eb   
                 3                  de3849188cb84f02bf780b556c9b02eb   
                 4                  de3849188cb84f02bf780b556c9b02eb   
a9e42c0917f8a29f 0                  1ef2fc386563f4bfb7f2cccc42a93edd   
                 1                  1ef2fc386563f4bfb7f2cccc42a93edd   
                 2                  1ef2fc386563f4bfb7f2cccc42a93edd   
                 3                  1ef2fc386563f4bfb7f2cccc42a93edd   
                 4                  1ef2fc386563f4bfb7f2cccc42a93edd   

                                                                                input  \
context.span_id  document_position                                                      
530af4522d6d8273 0                  What does the term 'Social Media' refer to in ...   
                 1                  What does the term 'Social Media' refer to in ...   
                 2                  What does the term 'Social Media' refer to in ...   
fbefcb4215e8cdc8 0                       List some social media guidelines for usage?   
                 1                       List some social media guidelines for usage?   
                 2                       List some social media guidelines for usage?   
ab83ed62670696ff 0                                    What are professional websites?   
                 1                                    What are professional websites?   
                 2                                    What are professional websites?   
fed4ecdba15db354 0                  What does the term 'Social Media' refer to in ...   
                 1                  What does the term 'Social Media' refer to in ...   
                 2                  What does the term 'Social Media' refer to in ...   
                 3                  What does the term 'Social Media' refer to in ...   
                 4                  What does the term 'Social Media' refer to in ...   
aa15c5e3b65359af 0                       List some social media guidelines for usage?   
                 1                       List some social media guidelines for usage?   
                 2                       List some social media guidelines for usage?   
                 3                       List some social media guidelines for usage?   
                 4                       List some social media guidelines for usage?   
a9e42c0917f8a29f 0                                    What are professional websites?   
                 1                                    What are professional websites?   
                 2                                    What are professional websites?   
                 3                         

In [26]:
hallucination_evaluator = HallucinationEvaluator(eval_model)
qa_correctness_evaluator = QAEvaluator(eval_model)
relevance_evaluator = RelevanceEvaluator(eval_model)

In [27]:
%%time
hallucination_eval_df, qa_correctness_eval_df = run_evals(
    dataframe=queries_df,
    evaluators=[hallucination_evaluator, qa_correctness_evaluator],
    provide_explanation=True,
    # concurrency=1
)
relevance_eval_df = run_evals(
    dataframe=retrieved_documents_df,
    evaluators=[relevance_evaluator],
    provide_explanation=True,
    # concurrency=1
)[0]

I0000 00:00:1737543835.687621    9601 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers


run_evals |          | 0/12 (0.0%) | ⏳ 00:00<? | ?it/s


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Provider List: https://docs.litellm.ai/docs/providers


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Provider List: https://docs.litellm.ai/docs/providers


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Provider List: https://docs.litellm.ai/docs/providers

Exception in worker on attempt 1: raised litellm.Timeout: APITimeoutError - Request timed out. Error_str: Request timed out.
Requeuing...
Exception in worker on attempt 1: raised litellm.Timeout: APITimeoutError - Request timed out. Error_str: Request timed out.
Requeuing...
Exception in worker on attempt 1: raised litellm.Timeout: APITimeoutError - Request timed out. Er

run_evals |          | 0/24 (0.0%) | ⏳ 00:00<? | ?it/s


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Provider List: https://docs.litellm.ai/docs/providers

Exception in worker on attempt 1: raised litellm.BadRequestError: Lm_studioException - Error code: 400 - {'error': 'The model has crashed without additional information. (Exit code: 11)'}
Requeuing...
CPU times: user 20.5 s, sys: 4.17 s, total: 24.7 s
Wall time: 51min 15s


In [28]:
px.Client().log_evaluations(
    SpanEvaluations(
        eval_name="Hallucination", dataframe=hallucination_eval_df
    ),
    SpanEvaluations(
        eval_name="QA Correctness", dataframe=qa_correctness_eval_df
    ),
    DocumentEvaluations(eval_name="Relevance", dataframe=relevance_eval_df),
)

## Query Optimization

In [10]:
from llama_index.core import SummaryIndex, Document
from llama_index.core.schema import IndexNode
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.agent import FunctionCallingAgent

In [11]:
doc_titles = [
    "Employee Communication Policy v1", "Phone_and_Internet_Allowance_Policy_abrd", "transport allowance"
]

In [13]:
# len(documents)

In [14]:
# documents[0]

In [15]:
# clean_text(documents[0].text)

### Recursive Retriever + Document Agents

In [16]:
title_docs = {}
for title in tqdm(doc_titles):
    parser = PyMuPDFReader()
    file_extractor = {".pdf": parser}
    doc_pages = SimpleDirectoryReader(
        input_files=[f"data/{title}.pdf"], file_extractor=file_extractor, encoding="utf8"
    ).load_data()

    doc_pages_ = []
    for page in doc_pages:
        # page.metadata["updated_at"] = str(datetime.datetime.now())  # add new metadata
    
        cleaned = clean_text(page.text)
    
        doc = Document(
            text=cleaned,
            metadata=page.metadata,
            id_=page.id_
        )  # Replace existing document object with cleaned text, keep existing metadata
        doc_pages_.append(doc)

    title_docs[title] = doc_pages_

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 17.45it/s]


In [17]:
title_docs["transport allowance"]

[Document(id_='33a83504-bd8c-4bd7-bd29-0d614574bec4', embedding=None, metadata={'file_path': 'data/transport allowance.pdf', 'file_name': 'transport allowance.pdf', 'file_type': 'application/pdf', 'file_size': 109667, 'creation_date': '2025-02-10', 'last_modified_date': '2025-01-30', 'total_pages': 3, 'source': '1'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='One Global Medical Technology Ltd. RC: 1264820 MEMO TO: Chief Operating Officer (COO), Finance Team, HR Team FROM: Head Human Resources DATE: 24th August 2023 SUBJECT: Lagos Team Transport Allowance (Update on Travel Policy for Lagos Team Members) This memo is being developed in response to the recent hike in transportation and petrol costs in Nigeria and is to apply to Helium Team Members who operate hybrid workdays of a minimum of 2 days in the office. As at the time of

In [18]:
# Build agents dictionary
agents = {}

for title in tqdm(doc_titles):
    # build vector index
    vector_index = VectorStoreIndex.from_documents(
        title_docs[title],
    )
    # build summary index
    summary_index = SummaryIndex.from_documents(
        title_docs[title],
    )
    # define query engines
    vector_query_engine = vector_index.as_query_engine()
    list_query_engine = summary_index.as_query_engine()

    # define tools
    query_engine_tools = [
        QueryEngineTool(
            query_engine=vector_query_engine,
            metadata=ToolMetadata(
                name=f"{title.replace(' ', '')}_vector_tool",
                description=(
                    f"Useful for retrieving specific context from {title}"
                ),
            ),
        ),
        QueryEngineTool(
            query_engine=list_query_engine,
            metadata=ToolMetadata(
                name=f"{title.replace(' ', '')}_summary_tool",
                description=(
                    "Useful for summarization questions related to"
                    f" {title}"
                ),
            ),
        ),
    ]

    # build agent
    agent = FunctionCallingAgent.from_tools(
        query_engine_tools,
        llm=llm,
        verbose=True,
    )

    agents[title] = agent

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:19<00:00,  6.41s/it]


In [35]:
summary_index

In [38]:
summary_index.as_query_engine().query("Write a one sentence summary about the contents of the document")

Response(response='The document outlines a new transport allowance policy for Lagos-based employees of One Global Medical Technology Ltd., effective from September 1st, 2023, to subsidize their commute to the office.', source_nodes=[NodeWithScore(node=TextNode(id_='5591597d-5d76-40f3-a251-a49755e660bb', embedding=None, metadata={'file_path': 'data/transport allowance.pdf', 'file_name': 'transport allowance.pdf', 'file_type': 'application/pdf', 'file_size': 109667, 'creation_date': '2025-02-10', 'last_modified_date': '2025-01-30', 'total_pages': 3, 'source': '1'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='33a83504-bd8c-4bd7-bd29-0d614574bec4', node_type='4', metadata={'file_path': 'data/transport allowance.pdf', 'file_name': 'transport allowance.pdf', 'file_type': 'application/pdf', 'file_size': 109667, 'creation_date': '2025-02-10', 'last_modified_date': '2025-01-30', 'total_pages': 3, 'source

In [41]:
vector_query_engine.query("Write one sentence about the contents of the document")

Response(response='The document outlines the terms and conditions for the transport allowance policy at One Global Medical Technology Ltd., including eligibility criteria, compliance requirements, and disciplinary actions for non-compliance.', source_nodes=[NodeWithScore(node=TextNode(id_='5cfbda08-7c40-4bf2-9e84-e0153aed2673', embedding=None, metadata={'file_path': 'data/transport allowance.pdf', 'file_name': 'transport allowance.pdf', 'file_type': 'application/pdf', 'file_size': 109667, 'creation_date': '2025-02-10', 'last_modified_date': '2025-01-30', 'total_pages': 3, 'source': '3'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='1c72cfcd-1567-4608-8df1-d43b33f80e4f', node_type='4', metadata={'file_path': 'data/transport allowance.pdf', 'file_name': 'transport allowance.pdf', 'file_type': 'application/pdf', 'file_size': 109667, 'creation_date': '2025-02-10', 'last_modified_date': '2025-01-30', 

In [19]:
agents

{'Employee Communication Policy v1': <llama_index.core.agent.function_calling.base.FunctionCallingAgent at 0x13e8cf650>,
 'Phone_and_Internet_Allowance_Policy_abrd': <llama_index.core.agent.function_calling.base.FunctionCallingAgent at 0x14bb43230>,
 'transport allowance': <llama_index.core.agent.function_calling.base.FunctionCallingAgent at 0x14bb427e0>}

In [20]:
objects = []
for title in tqdm(doc_titles):
    # define index node that links to these agents
    policy_summary = (
        f"This content contains policy documents about {title}. Use"
        " this index if you need to lookup specific facts about"
        f" {title}."
    )
    node = IndexNode(
        text=policy_summary, index_id=title, obj=agents[title]
    )
    objects.append(node)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 7349.83it/s]


In [21]:
# define top-level retriever
vector_index = VectorStoreIndex(
    objects=objects,
)
query_engine = vector_index.as_query_engine(similarity_top_k=1, verbose=True)

In [51]:
response = query_engine.query("How does an employee default on the transport allowance?")

Retrieval entering transport allowance: FunctionCallingAgent
Retrieving from object FunctionCallingAgent with query How does an employee default on the transport allowance?
> Running step b574541d-f9ce-4db2-b18f-683aa0f31120. Step input: How does an employee default on the transport allowance?
Added user message to memory: How does an employee default on the transport allowance?
=== LLM Response ===
<thinking> The User's question is about the specifics of how an employee might default on a transport allowance. To answer this, I need to retrieve the relevant context from the transport allowance tool. I will use the `transportallowance_vector_tool` to get the necessary information.</thinking>

=== Calling Function ===
Calling function: transportallowance_vector_tool with args: {"input": "default on transport allowance"}
=== Function Output ===
If an employee defaults on the transport allowance, they must reimburse the total amount of the allowance that was paid to them. This reimbursemen

In [52]:
# should use phone and internet policy document - What are the allowance types for phone and internet as well as the eligibility criteria?
response = query_engine.query("What are the allowance types for phone and internet as well as the eligibility criteria?")

Retrieval entering Phone_and_Internet_Allowance_Policy_abrd: FunctionCallingAgent
Retrieving from object FunctionCallingAgent with query What are the allowance types for phone and internet as well as the eligibility criteria?
> Running step 03b7fd3c-9568-4d61-be12-0d0d4e95f60e. Step input: What are the allowance types for phone and internet as well as the eligibility criteria?
Added user message to memory: What are the allowance types for phone and internet as well as the eligibility criteria?
=== LLM Response ===
<thinking> To provide the User with the information about the allowance types for phone and internet as well as the eligibility criteria, I need to use the available tools. I will first use the "Phone_and_Internet_Allowance_Policy_abrd_vector_tool" to retrieve specific context related to the allowance types and eligibility criteria. </thinking>

=== Calling Function ===
Calling function: Phone_and_Internet_Allowance_Policy_abrd_vector_tool with args: {"title": "allowance type

In [22]:
response = query_engine.query("Summarize the phone and internet policy?")

Retrieval entering Phone_and_Internet_Allowance_Policy_abrd: FunctionCallingAgent
Retrieving from object FunctionCallingAgent with query Summarize the phone and internet policy?
> Running step 848c5247-7a76-4051-966d-0dbb237b410b. Step input: Summarize the phone and internet policy?
Added user message to memory: Summarize the phone and internet policy?
=== LLM Response ===
<thinking> To summarize the phone and internet policy, I need to retrieve the relevant information from the available tools. I will use the "Phone_and_Internet_Allowance_Policy_abrd_summary_tool" to get a summary of the policy. </thinking>

=== Calling Function ===
Calling function: Phone_and_Internet_Allowance_Policy_abrd_summary_tool with args: {"input": "Phone and Internet Allowance Policy"}
=== Function Output ===
The Phone and Internet Allowance Policy outlines the provision of extra business-related phone and internet expenses to eligible employees. It includes two types of allowances: Monthly Phone and Interne

In [23]:
from llama_index.core.query_engine import SubQuestionQueryEngine

In [25]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=vector_index.as_query_engine(similarity_top_k=1, verbose=True),
        metadata=ToolMetadata(
            name="policy_engine",
            description="Context on various company policy documents",
        ),
    ),
]

sub_query_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=query_engine_tools,
    use_async=True,
    llm=llm
)

In [26]:
response = sub_query_engine.query("How does an employee default on the transport allowance?")

Generated 2 sub questions.
[policy_engine] Q: What is the transport allowance policy for employees?
[policy_engine] Q: What are the consequences of defaulting on the transport allowance policy?
Retrieval entering transport allowance: FunctionCallingAgent
> Running step 4c8bc6a6-3a45-44e3-932f-5b125af649af. Step input: What is the transport allowance policy for employees?
Added user message to memory: What is the transport allowance policy for employees?
Retrieval entering transport allowance: FunctionCallingAgent
> Running step 61a440a1-8161-4aaf-9288-f32c10d12b40. Step input: What are the consequences of defaulting on the transport allowance policy?
Added user message to memory: What are the consequences of defaulting on the transport allowance policy?


Retrying llama_index.llms.bedrock_converse.utils.converse_with_retry_async.<locals>._conversion_with_retry in 4.0 seconds as it raised ModelErrorException: An error occurred (ModelErrorException) when calling the Converse operation: The system encountered an unexpected error during processing. Try your request again..


=== LLM Response ===
<thinking> To provide the transport allowance policy for employees, I need to retrieve specific context from the transport allowance. The best approach is to use the `transportallowance_vector_tool` with the appropriate input. </thinking>

=== Calling Function ===
Calling function: transportallowance_vector_tool with args: {"input": "transport allowance policy for employees"}
=== Function Output ===
The transport allowance policy for employees is designed to subsidize transportation costs for eligible staff. Specifically, it applies to Lagos-based employees who are not receiving any other form of transport allowance and are required to be physically present in the office. To be eligible, employees must meet the following criteria:

1. Based in Lagos, Nigeria.
2. Not receiving any other transport allowance.
3. Required to be present in the office.
4. Attending the office a minimum of two days per week consistently.

The company will pay a monthly flat fee to cover t

In [27]:
response = sub_query_engine.query("What are the social media guidelines?")

Generated 1 sub questions.
[policy_engine] Q: What are the social media guidelines?
Retrieval entering Phone_and_Internet_Allowance_Policy_abrd: FunctionCallingAgent
> Running step eff16209-f358-455b-8653-6eb37b4e4556. Step input: What are the social media guidelines?
Added user message to memory: What are the social media guidelines?
=== LLM Response ===
<thinking> The User is asking about social media guidelines. To provide an accurate response, I need to retrieve specific context from the Phone_and_Internet_Allowance_Policy_abrd. I will use the "Phone_and_Internet_Allowance_Policy_abrd_vector_tool" to get the relevant information.</thinking>

=== Calling Function ===
Calling function: Phone_and_Internet_Allowance_Policy_abrd_vector_tool with args: {"title": "social media guidelines"}
=== Function Output ===
The provided context does not contain any information regarding social media guidelines. Therefore, no details about social media guidelines can be provided based on the given co

In [34]:
response.source_nodes

[NodeWithScore(node=TextNode(id_='5eae0a1d-b8b4-4cc0-8c37-7b547244f40e', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text="Sub question: What are the social media guidelines?\nResponse: I'm sorry, but I cannot provide details about social media guidelines based on the given context. If you have any other questions or need information on a different topic, feel free to ask!", mimetype='text/plain', start_char_idx=None, end_char_idx=None, metadata_seperator='\n', text_template='{metadata_str}\n\n{content}'), score=None),
 NodeWithScore(node=TextNode(id_='4f592f30-dcaf-498d-b521-1cbc0508754e', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text="<thinking> The tool has returned that the provided context does not contain any information rega

In [59]:
# %pip install llama-index-question-gen-guidance


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [42]:
from llama_index.vector_stores.redis import RedisVectorStore
from redisvl.schema import IndexSchema

In [43]:
REDIS_URL = f"redis://localhost:6379"
VECTOR_INDEX_SCHEMA = dict(json.load(open("../custom_redis_vector_schema.json", "r")))

In [46]:
index_schema = VECTOR_INDEX_SCHEMA
index_schema["index"]["prefix"] = "Phone_and_Internet_Allowance_Policy_abrd"
print(f"index_schema: {index_schema}")

index_schema: {'index': {'name': 'gdrive', 'prefix': 'Phone_and_Internet_Allowance_Policy_abrd'}, 'fields': [{'type': 'tag', 'name': 'id'}, {'type': 'tag', 'name': 'doc_id'}, {'type': 'text', 'name': 'text'}, {'type': 'tag', 'name': 'doc_team'}, {'type': 'tag', 'name': 'file_name'}, {'type': 'vector', 'name': 'vector', 'attrs': {'dims': 1024, 'algorithm': 'hnsw', 'distance_metric': 'cosine'}}]}


In [48]:
VECTOR_STORE = RedisVectorStore(
    schema=IndexSchema.from_dict(index_schema),
    redis_url=REDIS_URL,
)

19:59:49 redisvl.index.index INFO   Index already exists, not overwriting.


In [50]:
vector_index = VectorStoreIndex.from_vector_store(
    VECTOR_STORE, embed_model=embedder
)

In [56]:
vector_index.index_id

'a0407c89-ccbb-47d5-b2f3-2bfa7a19d472'

In [55]:
vector_index.as_query_engine(llm=llm).query("hello")

Response(response='Empty Response', source_nodes=[], metadata=None)